In [1]:
from gerrychain import Graph

In [2]:
# Read New Mexico county graph from the json file "NM_county.json"
filename = 'NM_county.json'

# GerryChain has a built-in function for reading graphs of this type:
G = Graph.from_json( filename )

In [3]:
# Let's impose a 1% population deviation (+/- 0.5%)
deviation = 0.01

import math
k = 3          # number of districts
total_population = sum(G.nodes[node]['TOTPOP'] for node in G.nodes)

L = math.ceil((1-deviation/2)*total_population/k)
U = math.floor((1+deviation/2)*total_population/k)
print("Using L =",L,"and U =",U,"and k =",k)

Using L = 682962 and U = 689824 and k = 3


In [4]:
# A nice ordering of New Mexico's vertices (counties)
ordering = [18, 19, 26, 22, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 20, 21, 23, 24, 25, 27, 28, 29, 30, 31, 32]

In [5]:
# finds vertices reachable from j in G[S]
def reachable(G, j, S):
    
    if j not in S:
        return list()
    
    child = [ j ]
    R = [ j ]
    
    while child:
        parent = child
        child = list()
        for u in parent:
            for v in G.neighbors(u):
                if v in S and v not in R:
                    child.append(v)
                    R.append(v)
    return R 

In [6]:
import copy
import networkx as nx

# Enumerate districts in G[S] that contain F. 
#   Assumes G[F] is connected 
def enumerate_districts(G,L,U,S,F,all_districts,added):
    
    # if G[V\F] has component with population < L, then F should pick up this enclave
    VF = [i for i in G.nodes if i not in F]
    
    for component in nx.connected_components(G.subgraph(VF)):
        
        population = sum(G.nodes[i]['TOTPOP'] for i in component)
        
        if population < L:
            
            withinS = True
            for i in component:
                if i not in S:
                    withinS = False
            
            connected = nx.is_connected(G.subgraph(F+list(component)))
            
            if connected and withinS: # merge F and component
                F += list(component)
            else:
                return # break early due to p(enclave) < L, or enclave not in S    
    
    pF = sum(G.nodes[i]['TOTPOP'] for i in F)
    
    S = reachable(G, F[0], S)
    pS = sum(G.nodes[i]['TOTPOP'] for i in S)
    
    # population exceeds what is allowed, or cannot reach what is required
    if pF > U or pS < L:   
        return
            
    # population in [L,U] and first time seeing F
    if pF >= L and added:   
        #print("District with population =",pF,"is",F)
        all_districts.append(F)
                
        
    # pick a vertex b that belongs to N(F) and S *and has largest population*
    b = None
    for f in F:
        for v in G.neighbors(f):
            if v not in F and v in S:
                if b == None:
                    b = v
                elif G.nodes[v]['TOTPOP'] > G.nodes[b]['TOTPOP']:
                    b = v
                  
    # no vertex b available to add
    if b is None:
        return
        
    # branch on vertex b
    
    # add b
    F_left = copy.deepcopy(F) + [b]
    S_left = copy.deepcopy(S)
    enumerate_districts(G,L,U,S_left,F_left,all_districts,True)

    # drop b
    F_right = copy.deepcopy(F)
    S_right = [i for i in S if i != b]
    enumerate_districts(G,L,U,S_right,F_right,all_districts,False)

In [7]:
n = len(ordering)
all_districts = list()

for p in range(n):
    
    j = ordering[p]
    V_j = ordering[slice(p,n,1)]
    S_j = reachable(G,j,V_j)
    districts_rooted_at_j = list()
    
    enumerate_districts(G,L,U,S_j,[j],districts_rooted_at_j,True)
    
    print("Number of districts rooted at",j,"is",len(districts_rooted_at_j))
    
    all_districts += districts_rooted_at_j
    
print(all_districts)

Number of districts rooted at 18 is 3
Number of districts rooted at 19 is 4310
Number of districts rooted at 26 is 2899
Number of districts rooted at 22 is 2873
Number of districts rooted at 14 is 9
Number of districts rooted at 0 is 0
Number of districts rooted at 1 is 0
Number of districts rooted at 2 is 0
Number of districts rooted at 3 is 0
Number of districts rooted at 4 is 0
Number of districts rooted at 5 is 0
Number of districts rooted at 6 is 0
Number of districts rooted at 7 is 0
Number of districts rooted at 8 is 0
Number of districts rooted at 9 is 0
Number of districts rooted at 10 is 0
Number of districts rooted at 11 is 0
Number of districts rooted at 12 is 0
Number of districts rooted at 13 is 0
Number of districts rooted at 15 is 0
Number of districts rooted at 16 is 0
Number of districts rooted at 17 is 0
Number of districts rooted at 20 is 0
Number of districts rooted at 21 is 0
Number of districts rooted at 23 is 0
Number of districts rooted at 24 is 0
Number of dis

In [8]:
# Let's draw them on a map
import geopandas as gpd

In [9]:
# Read New Mexico county shapefile from "NM_county.shp"
filename = 'NM_county.shp'

# Read geopandas dataframe from file
df = gpd.read_file( filename )

In [10]:
def export_to_png(G, df, districts, filename):
    
    assignment = [ -1 for u in G.nodes ]
    
    for j in range(len(districts)):
        for i in districts[j]:
            geoID = G.nodes[i]["GEOID10"]
            for u in G.nodes:
                if geoID == df['GEOID10'][u]:
                    assignment[u] = j
    
    if min(assignment[v] for v in G.nodes) < 0:
        print("Error: did not assign all nodes in district map png.")
    else:
        df['assignment'] = assignment
        my_fig = df.plot(column='assignment').get_figure()
        RESIZE_FACTOR = 3
        my_fig.set_size_inches(my_fig.get_size_inches()*RESIZE_FACTOR)
        plt.axis('off')
        my_fig.savefig(filename)
        plt.close(my_fig)

In [11]:
import matplotlib.pyplot as plt
count = 0

all_plans = list()

for district1 in all_districts:
    if ordering[0] in district1:
        
        for district2 in all_districts:
            if ordering[1] in district2:
                
                if set(district1).isdisjoint(set(district2)):
                    
                    district3 = [i for i in G.nodes if i not in district1 and i not in district2]

                    population = sum(G.nodes[i]['TOTPOP'] for i in district3)
                    connected = nx.is_connected(G.subgraph(district3))

                    if population >= L and population <= U and connected:
                        
                        districts = [district1, district2, district3]
                        all_plans.append(districts)
                        
                        count += 1
                        filename = 'NM-' + str(f"{count:04d}") + '.png'
                        export_to_png(G, df, districts, filename)
            

In [12]:
# Then, use this tutorial to create the video:
#   https://www.youtube.com/watch?v=LmxaYwmewWs

In [13]:
print(all_plans)

[[[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 27, 1, 15, 23, 31, 24, 13], [0, 3, 4, 7, 10, 16, 17, 21, 22, 25, 26, 29, 32]], [[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 27, 1, 15, 23, 31, 24, 13, 0], [3, 4, 7, 10, 16, 17, 21, 22, 25, 26, 29, 32]], [[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 27, 1, 15, 23, 31, 24, 0], [3, 4, 7, 10, 13, 16, 17, 21, 22, 25, 26, 29, 32]], [[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 27, 1, 15, 23, 3, 13, 24], [0, 4, 7, 10, 16, 17, 21, 22, 25, 26, 29, 31, 32]], [[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 27, 1, 15, 23, 3, 13, 24, 0], [4, 7, 10, 16, 17, 21, 22, 25, 26, 29, 31, 32]], [[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 27, 1, 15, 23, 32, 24, 13], [0, 3, 4, 7, 10, 16, 17, 21, 22, 25, 26, 29, 31]], [[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 27, 1, 15, 23, 32, 24, 13, 0], [3, 4, 7, 10, 16, 17, 21, 22, 25, 26, 29, 31]], [[18, 14], [19, 6, 20, 2, 12, 30, 5, 11, 28, 8, 9, 1, 15, 23, 31, 16, 0, 32, 24], [3, 4, 